In [ ]:
import requests
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
from collections import defaultdict
# CSV 읽기
df = pd.read_csv('도로명필터링.csv')

# 1. 설정값
service_key = "58789b62af0cd3a650734b7011a4005381f84821ee32b63f81bf9969dcefd54e"
base_url = "https://apis.data.go.kr/1613000/AptListService3/getRoadnameAptList3"

# 2. 작업을 수행할 단위 함수 (한 지역/도로명에 대한 수집 로직 격리)
def fetch_road_data(args):
    sgg, doro, name = args  # 인자 언패킹
    
    road_code = str(sgg) + str(doro)
    print(f"🚀 [{name}] 수집 시작 (Code: {road_code})")
    
    all_items = []
    page = 1
    
    while True:
        params = {
            'serviceKey': service_key,
            'roadCode' : road_code,
            'pageNo': page,
            'numOfRows': 100,
        }

        try:
            response = requests.get(base_url, params=params, timeout=10)
            if response.status_code != 200:
                print(f"❌ [{name}] 요청 실패 (Page {page}): Code {response.status_code}")
                break
                
            try:
                data = response.json()
            except Exception:
                # JSON 파싱 실패 시 (XML 등) 처리
                print(f"⚠️ [{name}] JSON 파싱 실패. 응답 내용 확인 필요.")
                break
            
            # 데이터 구조 방어 로직
            if 'response' not in data or 'body' not in data['response']:
                 # 데이터가 없거나 에러 메시지가 온 경우
                 break

            body = data['response']['body']
            
            # 1. items 가져오기 (없으면 None)
            items = body.get('items') 
            
            # 2. 데이터 형태 정규화 (무조건 리스트로 만듦)
            if items is None:
                items = []  # 데이터 없음
            elif isinstance(items, dict):
                items = [items] # 딕셔너리라면 리스트로 감싸기
            
            # 3. (중요) items가 리스트여도 내용물이 없는 경우 체크
            if len(items) == 0:
                # totalCount는 1인데 items가 비어있는 이상한 경우가 간혹 있음
                pass 
            
            # 데이터 저장
            all_items.extend(items)
            print(items)
            print(f" -> 가져온 개수: {len(items)} / 누적: {len(all_items)}")
            
            # 진행 상황 로그 (너무 많으면 주석 처리)
            # print(f" -> [{name}] {page}페이지 완료 ({len(all_items)}/{total_count})")
            
            # [탈출 조건]
            break
            
        except Exception as e:
            print(f"⚠️ [{name}] 에러 발생: {e}")
            break
    
    print(f"✅ [{name}] 수집 완료 (총 {len(all_items)}개)")
    return name, all_items

# 3. 메인 실행부 (Multi-threading 적용)
final_result = defaultdict(list)

# 작업할 데이터 리스트 준비 (zip으로 묶음)
tasks = list(zip(df['시군구'], df['도로명코드'], df['시군구명']))

# max_workers: 동시에 실행할 쓰레드 수. 
# 공공데이터 포털은 너무 빠르면 IP 차단되거나 에러를 뱉으므로 5~10 정도로 조절하세요.
MAX_WORKERS = 16

print(f"데이터 수집 시작... (총 {len(tasks)}건, 쓰레드 {MAX_WORKERS}개)")

with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
    # executor.map을 쓰면 순서대로 나오지만, 
    # as_completed를 쓰면 먼저 끝나는 놈부터 결과를 받습니다 (더 효율적)
    future_to_road = {executor.submit(fetch_road_data, task): task for task in tasks}
    
    for future in as_completed(future_to_road):
        try:
            name, items = future.result()
            final_result[name].extend(items)
        except Exception as exc:
            print(f"💥 치명적 에러 발생: {exc}")

print("\n🎉 전체 작업 종료")
print(f"수집된 지역 수: {len(final_result)}")

데이터 수집 시작... (총 3742건, 쓰레드 16개)
🚀 [노원구] 수집 시작 (Code: 113501000027)
🚀 [노원구] 수집 시작 (Code: 113503000001)
🚀 [노원구] 수집 시작 (Code: 113503000001)
🚀 [노원구] 수집 시작 (Code: 113503000001)
🚀 [노원구] 수집 시작 (Code: 113503000001)
🚀 [노원구] 수집 시작 (Code: 113503000001)
🚀 [노원구] 수집 시작 (Code: 113503000004)
🚀 [노원구] 수집 시작 (Code: 113503000004)
🚀 [노원구] 수집 시작 (Code: 113503000004)
🚀 [노원구] 수집 시작 (Code: 113503000115)
🚀 [노원구] 수집 시작 (Code: 113503000115)
🚀 [노원구] 수집 시작 (Code: 113503005046)
🚀 [노원구] 수집 시작 (Code: 113503005046)
🚀 [노원구] 수집 시작 (Code: 113503005046)
🚀 [노원구] 수집 시작 (Code: 113503005047)
🚀 [노원구] 수집 시작 (Code: 113503005047)
[{'kaptCode': 'A10023990', 'kaptName': '청년주택 와이엔타워', 'doroJuso': '서울특별시 노원구 공릉동 동일로 1000'}, {'kaptCode': 'A10027622', 'kaptName': '노원프레미어스엠코', 'doroJuso': '서울특별시 노원구 공릉동 동일로 986'}, {'kaptCode': 'A13920706', 'kaptName': '상계주공4단지', 'doroJuso': '서울특별시 노원구 상계동 동일로 1362'}, {'kaptCode': 'A13920707', 'kaptName': '상계주공6단지', 'doroJuso': '서울특별시 노원구 상계동 동일로 1390'}, {'kaptCode': 'A13971502', 'kaptName': '상계주공3단지', 'd

In [ ]:
rows = []

# 1. 딕셔너리 평탄화 (Flattening)
# key(지역명)와 value(아파트 리스트)를 분리해서 하나의 리스트로 만듭니다.
for region_name, items in final_result.items():
    for item in items:
        # 원본 훼손 방지를 위해 copy 사용 (선택 사항)
        row = item.copy()
        row['수집지역'] = region_name  # key값도 컬럼으로 추가 (필요 시)
        rows.append(row)

# 2. DataFrame 생성
df_result = pd.DataFrame(rows)

# 3. 중복 제거 (핵심)
# API 호출 시 페이지가 겹치거나 로직상 중복 수집된 건들을 'kaptCode'(단지코드) 기준으로 제거합니다.
# subset=['kaptCode']를 지정하면 코드가 같은 건 다 중복으로 봅니다.
df_clean = df_result.drop_duplicates(subset=['kaptCode'], keep='first')

# 4. 인덱스 재설정
df_clean = df_clean.reset_index(drop=True)

# 결과 확인
print(f"📌 원본 데이터 개수: {len(df_result)}")
print(f"✨ 정규화 후 개수: {len(df_clean)}")
print("\n[상위 5개 데이터]")
print(df_clean.head())

# 5. 파일 저장 (최종)
df_clean.to_csv('아파트_수집_최종.csv', index=False, encoding='utf-8-sig')
# df_clean.to_excel('아파트_수집_최종.xlsx', index=False)

📌 원본 데이터 개수: 21
✨ 정규화 후 개수: 7

[상위 5개 데이터]
    kaptCode    kaptName                doroJuso 수집지역
0  A10023990  청년주택 와이엔타워  서울특별시 노원구 공릉동 동일로 1000  노원구
1  A10027622   노원프레미어스엠코   서울특별시 노원구 공릉동 동일로 986  노원구
2  A13920706     상계주공4단지  서울특별시 노원구 상계동 동일로 1362  노원구
3  A13920707     상계주공6단지  서울특별시 노원구 상계동 동일로 1390  노원구
4  A13971502     상계주공3단지  서울특별시 노원구 상계동 동일로 1389  노원구
